# 衝突回避 - モデルの学習 (ResNet18モデルの学習)

このノートブックでは、衝突回避のために画像を``free「直進する」``と``blocked「旋回する」``の2つのクラスに分類するようにモデルをトレーニングします。

今回は人気のあるモデルのResNet18をベースに学習します。

## それでは、始めましょう
まずは必要なライブラリを読み込みます。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### データセットインスタンスを作成

[torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)パッケージに含まれる``ImageFolder`` classを使用します。学習用のデータを準備するために、``torchvision.transforms``パッケージを使って画像変換を定義します。

In [ ]:
dataset = datasets.ImageFolder(
    '../dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### データセットをトレーニングセットとテストセットに分割する

次に、データセットを*トレーニング用*と*テスト用*のデータセットに分割します。この例では、*トレーニング用*に90%, *テスト用*に10%で分けます。*テスト用*のデータセットは、学習中にモデルの精度を検証するために使用されます。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### バッチ処理で学習データとテストデータを読み込むためのデータローダーを作成

[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)クラスは、モデル学習中に次のデータ処理が完了出来るようにサブプロセスで並列処理にして実装します。  
データのシャッフル、バッチでのデータロードのために使用します。この例では、1回のバッチ処理で8枚の画像を使用します。これをバッチサイズと呼び、GPUのメモリ使用量と、モデルの精度に影響を与えます。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### JetBot用にモデルを変更する

torchvisionで使用可能なImageNetデータセットで学習済みのResNet18モデルを使用します。

*転移学習*と呼ばれる手法で、すでに画像分類できる特徴を持つニューラルネットワーク層を、別の目的のために作られたモデルに適用することで、短時間で良好な結果を得られるモデルを作成することができます。

ResNet18の詳細: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

転移学習の詳細：https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
model = models.resnet18(pretrained=True)
# モデルを凍結して使う場合は、全てのパラメータが持つ学習フラグを無効化します。デフォルトはrequires_grad = Trueで全てのパラメータを再学習します。
#for param in model.parameters():
#    param.requires_grad = False

ResNet18モデルはImageNetを学習するために作られているため、1000種類の画像分類が可能な出力を持っています。ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、JetBotで欲しい出力``free``と``blocked``の2種類を得られるモデル構造にします。

In [ ]:
print(model.fc.in_features)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

デフォルトではモデルのweightはCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

# 視覚化ユーティリティ
[bokeh](https://docs.bokeh.org/en/latest/docs/installation.html)を使って学習中の損失(loss)と精度(accuracy)をグラフに表示することができます。

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tickers import SingleIntervalTicker
output_notebook()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

p1 = figure(title="Loss", x_axis_label="Epoch", plot_height=300, plot_width=360)
p2 = figure(title="Accuracy", x_axis_label="Epoch", plot_height=300, plot_width=360)

source1 = ColumnDataSource(data={'epochs': [], 'trainlosses': [], 'testlosses': [] })
source2 = ColumnDataSource(data={'epochs': [], 'train_accuracies': [], 'test_accuracies': []})

#r = p1.multi_line(ys=['trainlosses', 'testlosses'], xs='epochs', color=colors, alpha=0.8, legend_label=['Training','Test'], source=source)
r1 = p1.line(x='epochs', y='trainlosses', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source1)
r2 = p1.line(x='epochs', y='testlosses', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source1)

r3 = p2.line(x='epochs', y='train_accuracies', line_width=2, color=colors[0], alpha=0.8, legend_label="Train", source=source2)
r4 = p2.line(x='epochs', y='test_accuracies', line_width=2, color=colors[1], alpha=0.8, legend_label="Test", source=source2)

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2.legend.location = "bottom_right"
p2.legend.click_policy="hide"

### モデルの学習

10エポック学習し、各エポックで以前の最高精度と現在の精度を比較することにより、最高精度を更新した場合に保存します。  
現在の精度が以前の最高精度と等しい場合は、損失の少ない方を保存します。

> 1エポックは、私たちが用意したトレーニング用のデータ全部を1回学習することです。一度に8枚の画像を学習するミニバッチ処理を複数回実行することで1エポックが完了します。

In [ ]:
NUM_EPOCHS = 10
BEST_MODEL_PATH = 'best_model_resnet18.pth'
best_accuracy = 0.0
saved_loss = 1e9

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

handle = show(row(p1, p2), notebook_handle=True)

for epoch in range(NUM_EPOCHS):

    train_loss = 0.0 # for graph
    train_error_count = 0.0 # for graph
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        train_loss += loss # for graph
        train_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1)))) # for graph
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader) # for graph

    test_loss = 0.0 # for graph
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_loss += loss # for graph
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    test_loss /= len(test_loader) # for graph

    train_accuracy = 1.0 - float(train_error_count) / float(len(train_dataset)) # for graph
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))

    # 今回のepoch学習のテスト結果がよければ保存します
    is_saved = False
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy
        saved_loss = test_loss
        is_saved = True
    elif test_accuracy == best_accuracy:
        if test_loss < saved_loss:
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            saved_loss = test_loss
            is_saved = True

    print('%d: %f, %f, %f, %f, ' % (epoch+1, train_loss, test_loss, train_accuracy, test_accuracy)+("saved" if is_saved else "not saved"))

    # 学習状況をグラフに表示します
    new_data1 = {'epochs': [epoch+1],
                 'trainlosses': [float(train_loss)],
                 'testlosses': [float(test_loss)] }
    source1.stream(new_data1)
    new_data2 = {'epochs': [epoch+1],
                 'train_accuracies': [float(train_accuracy)],
                 'test_accuracies': [float(test_accuracy)] }
    source2.stream(new_data2)
    push_notebook(handle=handle)

学習が完了すると、``live_demo_resnet18.ipynb``で推論に使う``best_model_resnet18.pth``が生成されます。

## Next(次)

次は、`live_demo_resnet18.ipynb`を実行します。  
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。

TensorRTを試したい人は、trtフォルダの中の``convert_to_trt.ipynb``を実行し、学習済みモデルをTensorRT形式に変換し、``live_demo_resnet18_trt.ipynb``を実行し自動走行します。


[trt/convert_trt.ipynb](./trt/convert_trt.ipynb)をクリックし、TensorRTに変換しましょう。